![](https://images.unsplash.com/photo-1602084551218-a28205125639?ixlib=rb-1.2.1&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=2070&q=80)

<div class = 'alert alert-block alert-info'
     style = 'background-color:#4c1c84;
              color:#eeebf1;
              border-width:5px;
              border-color:#4c1c84;
              font-family:Comic Sans MS;
              border-radius: 50px 50px'>
    <p style = 'font-size:24px'>Exp 021</p>
    <a href = "#Config"
       style = "color:#eeebf1;
                font-size:14px">1.Config</a><br>
    <a href = "#Settings"
       style = "color:#eeebf1;
                font-size:14px">2.Settings</a><br>
    <a href = "#Data-Load"
       style = "color:#eeebf1;
                font-size:14px">3.Data Load</a><br>
    <a href = "#Pytorch-Settings"
       style = "color:#eeebf1;
                font-size:14px">4.Pytorch Settings</a><br>
    <a href = "#Training"
       style = "color:#eeebf1;
                font-size:14px">5.Training</a><br>
</div>

<p style = 'font-size:24px;
            color:#4c1c84'>
    実施したこと
</p>
    <li style = "color:#4c1c84;
                font-size:14px">過去コンペの予測ラベルを最適化</li>
    <li style = "color:#4c1c84;
                font-size:14px">Ridge</li>

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Config
</h1>
<br>

In [1]:
import sys
sys.path.append("../src/utils/iterative-stratification/")
sys.path.append("../src/utils/detoxify")

In [2]:
import warnings
warnings.simplefilter('ignore')

import os
import gc
gc.enable()
import sys
import glob
import copy
import math
import time
import random
import string
import psutil
import pathlib
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict
from box import Box
from typing import Optional
from pprint import pprint

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib

from tqdm.auto import tqdm as tqdmp
from tqdm.autonotebook import tqdm as tqdm
tqdmp.pandas()

## Model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, AdamW
from transformers import RobertaModel, RobertaForSequenceClassification
from transformers import RobertaTokenizer
from transformers import LukeTokenizer, LukeModel, LukeConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification


# Pytorch Lightning
import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning import LightningDataModule, LightningDataModule
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.loggers.csv_logs import CSVLogger
from pytorch_lightning.callbacks import RichProgressBar

from sklearn.linear_model import Ridge
from sklearn.svm import SVC, SVR
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.stats import rankdata
from cuml.svm import SVR
from cuml.linear_model import Ridge as cuml_Ridge
import cudf
from detoxify import Detoxify


2022-01-20 05:36:22.585512: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
import torch

config = {
    "seed": 42,
    "root": "/content/drive/MyDrive/kaggle/Jigsaw/raw",
    "n_fold": 5,
    "epoch": 5,
    "max_length": 128,
    "environment": "AWS",
    "project": "Jigsaw",
    "entity": "dataskywalker",
    "exp_name": "021_exp",
    "margin": 0.5,
    "train_fold": [0, 1, 2, 3, 4],

    "trainer": {
        "gpus": 1,
        "accumulate_grad_batches": 8,
        "progress_bar_refresh_rate": 1,
        "fast_dev_run": False,
        "num_sanity_val_steps": 0,
    },

    "train_loader": {
        "batch_size": 32,
        "shuffle": True,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": True,
    },

    "valid_loader": {
        "batch_size": 64,
        "shuffle": False,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": False,
    },

    "test_loader": {
        "batch_size": 32,
        "shuffle": False,
        "num_workers": 1,
        "pin_memory": True,
        "drop_last": False,
    },

    "backbone": {
        "name": "studio-ousia/luke-base",
        "output_dim": 1,
    },

    "optimizer": {
        "name": "torch.optim.AdamW",
        "params": {
            "lr": 1e-6,
        },
    },

    "scheduler": {
        "name": "torch.optim.lr_scheduler.CosineAnnealingWarmRestarts",
        "params": {
            "T_0": 20,
            "eta_min": 0,
        },
    },

    "loss": "nn.MarginRankingLoss",
}

config = Box(config)
pprint(config)

{'backbone': <Box: {'name': 'studio-ousia/luke-base', 'output_dim': 1}>,
 'entity': 'dataskywalker',
 'environment': 'AWS',
 'epoch': 5,
 'exp_name': '021_exp',
 'loss': 'nn.MarginRankingLoss',
 'margin': 0.5,
 'max_length': 128,
 'n_fold': 5,
 'optimizer': <Box: {'name': 'torch.optim.AdamW', 'params': {'lr': 1e-06}}>,
 'project': 'Jigsaw',
 'root': '/content/drive/MyDrive/kaggle/Jigsaw/raw',
 'scheduler': {'name': 'torch.optim.lr_scheduler.CosineAnnealingWarmRestarts',
               'params': <Box: {'T_0': 20, 'eta_min': 0}>},
 'seed': 42,
 'test_loader': {'batch_size': 32,
                 'drop_last': False,
                 'num_workers': 1,
                 'pin_memory': True,
                 'shuffle': False},
 'train_fold': <BoxList: [0, 1, 2, 3, 4]>,
 'train_loader': {'batch_size': 32,
                  'drop_last': True,
                  'num_workers': 1,
                  'pin_memory': True,
                  'shuffle': True},
 'trainer': {'accumulate_grad_batches': 8,
   

In [4]:
# 個人的にAWSやKaggle環境やGoogle Colabを行ったり来たりしているのでまとめています
import os
import sys
from pathlib import Path

if config.environment == 'AWS':
    
    INPUT_DIR = Path('/mnt/work/data/kaggle/Jigsaw/')
    MODEL_DIR = Path(f'../models/{config.exp_name}/')
    OUTPUT_DIR = Path(f'../data/interim/{config.exp_name}/')
    UTIL_DIR = Path('/mnt/work/shimizu/kaggle/PetFinder/src/utils')
    
    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    print(f"Your environment is 'AWS'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}\nUTIL_DIR is {UTIL_DIR}")
    
    
elif config.environment == 'Kaggle':
    INPUT_DIR = Path('../input/*****')
    MODEL_DIR = Path('./')
    OUTPUT_DIR = Path('./')
    print(f"Your environment is 'Kaggle'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}")

    
elif config.environment == 'Colab':
    INPUT_DIR = Path('/content/drive/MyDrive/kaggle/Jigsaw/raw')
    BASE_DIR = Path("/content/drive/MyDrive/kaggle/Jigsaw/interim")

    MODEL_DIR = BASE_DIR / f'{config.exp_name}'
    OUTPUT_DIR = BASE_DIR / f'{config.exp_name}/'

    os.makedirs(MODEL_DIR, exist_ok=True)
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    if not os.path.exists(INPUT_DIR):
        print('Please Mount your Google Drive.')
    else:
        print(f"Your environment is 'Colab'.\nINPUT_DIR is {INPUT_DIR}\nMODEL_DIR is {MODEL_DIR}\nOUTPUT_DIR is {OUTPUT_DIR}")
        
else:
    print("Please choose 'AWS' or 'Kaggle' or 'Colab'.\nINPUT_DIR is not found.")

Your environment is 'AWS'.
INPUT_DIR is /mnt/work/data/kaggle/Jigsaw
MODEL_DIR is ../models/021_exp
OUTPUT_DIR is ../data/interim/021_exp
UTIL_DIR is /mnt/work/shimizu/kaggle/PetFinder/src/utils


In [5]:
# Seed固定
seed_everything(config.seed)

42

In [6]:
## 処理時間計測
@contextmanager
def timer(name:str, slack:bool=False):
    t0 = time.time()
    p = psutil.Process(os.getpid())
    m0 = p.memory_info()[0] / 2. ** 30
    print(f'<< {name} >> Start')
    yield
    
    m1 = p.memory_info()[0] / 2. ** 30
    delta = m1 - m0
    sign = '+' if delta >= 0 else '-'
    delta = math.fabs(delta)
    
    print(f"<< {name} >> {m1:.1f}GB({sign}{delta:.1f}GB):{time.time() - t0:.1f}sec", file=sys.stderr)

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Data Load
</h1>
<br>

In [7]:
## Data Check
for dirnames, _, filenames in os.walk(INPUT_DIR):
    
    for filename in filenames:

        print(f'{dirnames}/{filename}')

/mnt/work/data/kaggle/Jigsaw/comments_to_score.csv
/mnt/work/data/kaggle/Jigsaw/sample_submission.csv
/mnt/work/data/kaggle/Jigsaw/validation_data.csv


In [8]:
val_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/validation_data.csv")
test_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/comments_to_score.csv")

display(val_df.head())
display(test_df.head())

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
    Detoxify
</h1>
<br>

In [14]:
detoxi_df = pd.DataFrame()

<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Original
</h2>
<br>

In [9]:
loaded = torch.load("../data/external/detoxify_ckpt/toxic_original-c1212f89.ckpt")
loaded["config"]["arch"]["args"]

{'num_classes': 6,
 'model_type': 'bert-base-uncased',
 'model_name': 'BertForSequenceClassification',
 'tokenizer_name': 'BertTokenizer'}

In [10]:
bert_tokeninzer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_tokeninzer.save_pretrained("../data/processed/bert-base-uncased")
bert_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
bert_model.save_pretrained('../data/processed/bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
detox_model = Detoxify(
    'original', 
    checkpoint="../data/external/detoxify_ckpt/toxic_original-c1212f89.ckpt",
    device="cuda"
)
detox_model.predict(val_df["more_toxic"].tolist()[0])

{'toxicity': 0.91618466,
 'severe_toxicity': 0.005879892,
 'obscene': 0.3734918,
 'threat': 0.0024057706,
 'insult': 0.20671596,
 'identity_attack': 0.0027373598}

In [13]:
val_df['more_detoxify_score'] = val_df['more_toxic'].progress_map(lambda line: detox_model.predict(line))
val_df['less_detoxify_score'] = val_df['less_toxic'].progress_map(lambda line: detox_model.predict(line))

more_detoxify_df = val_df["more_detoxify_score"].apply(pd.Series)
less_detoxify_df = val_df["less_detoxify_score"].apply(pd.Series)

display(more_detoxify_df.head())
display(less_detoxify_df.head())

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032


,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,0.987555,0.053543,0.924013,0.002496,0.188302,0.002181
1,0.001304,0.000096,0.000178,0.000102,0.000191,0.000156
2,0.071497,0.000298,0.001521,0.000425,0.002012,0.004720
3,0.933284,0.003731,0.048126,0.013924,0.366017,0.005633
4,0.972585,0.030403,0.709035,0.002615,0.831553,0.059486


In [16]:
detoxi_df = pd.concat([
    detoxi_df,
    more_detoxify_df.add_prefix("ori_bert_more__")
], axis=1)

detoxi_df = pd.concat([
    detoxi_df,
    less_detoxify_df.add_prefix("ori_bert_less__")
], axis=1)

detoxi_df.head()

,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,ori_bert_less__severe_toxicity,ori_bert_less__obscene,ori_bert_less__threat,ori_bert_less__insult,ori_bert_less__identity_attack
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,0.053543,0.924013,0.002496,0.188302,0.002181
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,0.000096,0.000178,0.000102,0.000191,0.000156
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,0.000298,0.001521,0.000425,0.002012,0.004720
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,0.003731,0.048126,0.013924,0.366017,0.005633
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,0.030403,0.709035,0.002615,0.831553,0.059486


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Unbiased
</h2>
<br>

In [17]:
loaded = torch.load("../data/external/detoxify_ckpt/toxic_debiased-c7548aa0.ckpt")
loaded["config"]["arch"]["args"]

{'num_classes': 16,
 'model_type': 'roberta-base',
 'model_name': 'RobertaForSequenceClassification',
 'tokenizer_name': 'RobertaTokenizer'}

In [18]:
bert_tokeninzer = RobertaTokenizer.from_pretrained('roberta-base')
bert_tokeninzer.save_pretrained(f"../data/processed/roberta-base")
bert_model = RobertaForSequenceClassification.from_pretrained('roberta-base')
bert_model.save_pretrained('../data/processed/roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [19]:
detox_model = Detoxify(
    'unbiased', 
    checkpoint="../data/external/detoxify_ckpt/toxic_debiased-c7548aa0.ckpt",
    device="cuda"
)
detox_model.predict(val_df["more_toxic"].tolist()[0])

{'toxicity': 0.8752487,
 'severe_toxicity': 0.0029985397,
 'obscene': 0.26864234,
 'identity_attack': 0.013699602,
 'insult': 0.1651928,
 'threat': 0.0009332173,
 'sexual_explicit': 0.68039703}

In [20]:
val_df['more_detoxify_score'] = val_df['more_toxic'].progress_map(lambda line: detox_model.predict(line))
val_df['less_detoxify_score'] = val_df['less_toxic'].progress_map(lambda line: detox_model.predict(line))

more_detoxify_df = val_df["more_detoxify_score"].apply(pd.Series)
less_detoxify_df = val_df["less_detoxify_score"].apply(pd.Series)

display(more_detoxify_df.head())
display(less_detoxify_df.head())

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.875249,0.002999,0.268642,0.013700,0.165193,0.000933,0.680397
1,0.866404,0.003731,0.532514,0.007912,0.304798,0.003272,0.331255
2,0.640468,0.000956,0.061516,0.005102,0.007513,0.000648,0.683325
3,0.996822,0.192608,0.966310,0.063918,0.906610,0.004926,0.939895
4,0.930674,0.000983,0.036329,0.504768,0.741871,0.001739,0.001975


,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.971695,0.008350,0.835039,0.005941,0.487024,0.000762,0.032691
1,0.020485,0.000005,0.000916,0.000352,0.004296,0.000057,0.000120
2,0.350858,0.000018,0.000236,0.164579,0.126239,0.000614,0.000131
3,0.689309,0.000326,0.087503,0.006940,0.390970,0.002517,0.002921
4,0.971044,0.013073,0.429244,0.074235,0.397570,0.001827,0.963226


In [21]:
detoxi_df = pd.concat([
    detoxi_df,
    more_detoxify_df.add_prefix("unbiased_roberta_more__")
], axis=1)

detoxi_df = pd.concat([
    detoxi_df,
    less_detoxify_df.add_prefix("unbiased_roberta_less__")
], axis=1)

detoxi_df.head()

,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,ori_bert_less__severe_toxicity,ori_bert_less__obscene,ori_bert_less__threat,...,unbiased_roberta_more__insult,unbiased_roberta_more__threat,unbiased_roberta_more__sexual_explicit,unbiased_roberta_less__toxicity,unbiased_roberta_less__severe_toxicity,unbiased_roberta_less__obscene,unbiased_roberta_less__identity_attack,unbiased_roberta_less__insult,unbiased_roberta_less__threat,unbiased_roberta_less__sexual_explicit
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,0.053543,0.924013,0.002496,...,0.165193,0.000933,0.680397,0.971695,0.008350,0.835039,0.005941,0.487024,0.000762,0.032691
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,0.000096,0.000178,0.000102,...,0.304798,0.003272,0.331255,0.020485,0.000005,0.000916,0.000352,0.004296,0.000057,0.000120
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,0.000298,0.001521,0.000425,...,0.007513,0.000648,0.683325,0.350858,0.000018,0.000236,0.164579,0.126239,0.000614,0.000131
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,0.003731,0.048126,0.013924,...,0.906610,0.004926,0.939895,0.689309,0.000326,0.087503,0.006940,0.390970,0.002517,0.002921
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,0.030403,0.709035,0.002615,...,0.741871,0.001739,0.001975,0.971044,0.013073,0.429244,0.074235,0.397570,0.001827,0.963226


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Multilingual
</h2>
<br>

In [22]:
loaded = torch.load("../data/external/detoxify_ckpt/multilingual_debiased-0b549669.ckpt")
loaded["config"]["arch"]["args"]

{'num_classes': 16,
 'model_type': 'xlm-roberta-base',
 'model_name': 'XLMRobertaForSequenceClassification',
 'tokenizer_name': 'XLMRobertaTokenizer'}

In [23]:
bert_tokeninzer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
bert_tokeninzer.save_pretrained(f"../data/processed/xlm-roberta-base")
bert_model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base')
bert_model.save_pretrained('../data/processed/xlm-roberta-base')

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [24]:
detox_model = Detoxify(
    'multilingual', 
    checkpoint="../data/external/detoxify_ckpt/multilingual_debiased-0b549669.ckpt",
    device="cuda"
)
detox_model.predict(val_df["more_toxic"].tolist()[0])

{'toxicity': 0.72579426,
 'severe_toxicity': 0.0041608843,
 'obscene': 0.11467257,
 'identity_attack': 0.006995354,
 'insult': 0.022113552,
 'threat': 0.0018297561,
 'sexual_explicit': 0.5801136}

In [25]:
val_df['more_detoxify_score'] = val_df['more_toxic'].progress_map(lambda line: detox_model.predict(line))
val_df['less_detoxify_score'] = val_df['less_toxic'].progress_map(lambda line: detox_model.predict(line))

more_detoxify_df = val_df["more_detoxify_score"].apply(pd.Series)
less_detoxify_df = val_df["less_detoxify_score"].apply(pd.Series)

display(more_detoxify_df.head())
display(less_detoxify_df.head())

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.725794,0.004161,0.114673,0.006995,0.022114,0.001830,0.580114
1,0.950540,0.010019,0.652214,0.005386,0.570360,0.004682,0.742930
2,0.598606,0.003005,0.049947,0.002181,0.010233,0.002453,0.674482
3,0.997031,0.336659,0.981065,0.020054,0.926189,0.106624,0.973957
4,0.963474,0.000439,0.004848,0.029107,0.684872,0.001914,0.000540


,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.985322,0.012029,0.339552,0.004669,0.255090,0.002174,0.060398
1,0.028772,0.000102,0.001581,0.001113,0.007293,0.000073,0.000062
2,0.876132,0.000052,0.000364,0.039193,0.665449,0.000510,0.000110
3,0.938025,0.009904,0.147672,0.006287,0.159671,0.008066,0.006064
4,0.944948,0.007452,0.073160,0.010256,0.156879,0.007220,0.942082


In [26]:
detoxi_df = pd.concat([
    detoxi_df,
    more_detoxify_df.add_prefix("multilingual_xlmroberta_more__")
], axis=1)

detoxi_df = pd.concat([
    detoxi_df,
    less_detoxify_df.add_prefix("multilingual_xlmroberta_less__")
], axis=1)

detoxi_df.head()

,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,ori_bert_less__severe_toxicity,ori_bert_less__obscene,ori_bert_less__threat,...,multilingual_xlmroberta_more__insult,multilingual_xlmroberta_more__threat,multilingual_xlmroberta_more__sexual_explicit,multilingual_xlmroberta_less__toxicity,multilingual_xlmroberta_less__severe_toxicity,multilingual_xlmroberta_less__obscene,multilingual_xlmroberta_less__identity_attack,multilingual_xlmroberta_less__insult,multilingual_xlmroberta_less__threat,multilingual_xlmroberta_less__sexual_explicit
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,0.053543,0.924013,0.002496,...,0.022114,0.001830,0.580114,0.985322,0.012029,0.339552,0.004669,0.255090,0.002174,0.060398
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,0.000096,0.000178,0.000102,...,0.570360,0.004682,0.742930,0.028772,0.000102,0.001581,0.001113,0.007293,0.000073,0.000062
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,0.000298,0.001521,0.000425,...,0.010233,0.002453,0.674482,0.876132,0.000052,0.000364,0.039193,0.665449,0.000510,0.000110
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,0.003731,0.048126,0.013924,...,0.926189,0.106624,0.973957,0.938025,0.009904,0.147672,0.006287,0.159671,0.008066,0.006064
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,0.030403,0.709035,0.002615,...,0.684872,0.001914,0.000540,0.944948,0.007452,0.073160,0.010256,0.156879,0.007220,0.942082


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Original-Small
</h2>
<br>

In [27]:
loaded = torch.load("../data/external/detoxify_ckpt/original-albert-0e1d6498.ckpt")
loaded["config"]["arch"]["args"]

{'num_classes': 6,
 'model_type': 'albert-base-v2',
 'model_name': 'AlbertForSequenceClassification',
 'tokenizer_name': 'AlbertTokenizer'}

In [31]:
from transformers import AlbertTokenizer, AlbertForSequenceClassification

bert_tokeninzer = AlbertTokenizer.from_pretrained('albert-base-v2')
bert_tokeninzer.save_pretrained(f"../data/processed/albert-base-v2")
bert_model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')
bert_model.save_pretrained('../data/processed/albert-base-v2')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [32]:
detox_model = Detoxify(
    'original-small', 
    checkpoint="../data/external/detoxify_ckpt/original-albert-0e1d6498.ckpt",
    device="cuda"
)
detox_model.predict(val_df["more_toxic"].tolist()[0])

{'toxicity': 0.92202544,
 'severe_toxicity': 0.005315636,
 'obscene': 0.2939583,
 'threat': 0.0027349656,
 'insult': 0.25835648,
 'identity_attack': 0.003065943}

In [33]:
val_df['more_detoxify_score'] = val_df['more_toxic'].progress_map(lambda line: detox_model.predict(line))
val_df['less_detoxify_score'] = val_df['less_toxic'].progress_map(lambda line: detox_model.predict(line))

more_detoxify_df = val_df["more_detoxify_score"].apply(pd.Series)
less_detoxify_df = val_df["less_detoxify_score"].apply(pd.Series)

display(more_detoxify_df.head())
display(less_detoxify_df.head())

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,0.922025,0.005316,0.293958,0.002735,0.258356,0.003066
1,0.840008,0.003191,0.093295,0.002742,0.436359,0.051488
2,0.034373,0.000140,0.004662,0.000273,0.001342,0.000335
3,0.998076,0.368854,0.989622,0.019516,0.920091,0.025175
4,0.896274,0.010004,0.070025,0.008721,0.235448,0.313888


,toxicity,severe_toxicity,obscene,threat,insult,identity_attack
0,0.928659,0.010311,0.800586,0.001779,0.235096,0.002898
1,0.010340,0.000074,0.001108,0.000119,0.000342,0.000183
2,0.040394,0.000093,0.001891,0.000172,0.001051,0.000526
3,0.849285,0.002945,0.019918,0.007902,0.101257,0.004247
4,0.929404,0.008560,0.574357,0.002146,0.632560,0.011569


In [34]:
detoxi_df = pd.concat([
    detoxi_df,
    more_detoxify_df.add_prefix("ori_albert_more__")
], axis=1)

detoxi_df = pd.concat([
    detoxi_df,
    less_detoxify_df.add_prefix("ori_albert_less__")
], axis=1)

detoxi_df.head()

,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,ori_bert_less__severe_toxicity,ori_bert_less__obscene,ori_bert_less__threat,...,ori_albert_more__obscene,ori_albert_more__threat,ori_albert_more__insult,ori_albert_more__identity_attack,ori_albert_less__toxicity,ori_albert_less__severe_toxicity,ori_albert_less__obscene,ori_albert_less__threat,ori_albert_less__insult,ori_albert_less__identity_attack
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,0.053543,0.924013,0.002496,...,0.293958,0.002735,0.258356,0.003066,0.928659,0.010311,0.800586,0.001779,0.235096,0.002898
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,0.000096,0.000178,0.000102,...,0.093295,0.002742,0.436359,0.051488,0.010340,0.000074,0.001108,0.000119,0.000342,0.000183
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,0.000298,0.001521,0.000425,...,0.004662,0.000273,0.001342,0.000335,0.040394,0.000093,0.001891,0.000172,0.001051,0.000526
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,0.003731,0.048126,0.013924,...,0.989622,0.019516,0.920091,0.025175,0.849285,0.002945,0.019918,0.007902,0.101257,0.004247
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,0.030403,0.709035,0.002615,...,0.070025,0.008721,0.235448,0.313888,0.929404,0.008560,0.574357,0.002146,0.632560,0.011569


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Unbiased-Small
</h2>
<br>

In [35]:
loaded = torch.load("../data/external/detoxify_ckpt/unbiased-albert-c8519128.ckpt")
loaded["config"]["arch"]["args"]

{'num_classes': 16,
 'model_type': 'albert-base-v2',
 'model_name': 'AlbertForSequenceClassification',
 'tokenizer_name': 'AlbertTokenizer'}

In [36]:
bert_tokeninzer = AlbertTokenizer.from_pretrained('albert-base-v2')
bert_tokeninzer.save_pretrained(f"../data/processed/albert-base-v2")
bert_model = AlbertForSequenceClassification.from_pretrained('albert-base-v2')
bert_model.save_pretrained('../data/processed/albert-base-v2')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.bias', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

In [37]:
detox_model = Detoxify(
    'unbiased-small', 
    checkpoint="../data/external/detoxify_ckpt/unbiased-albert-c8519128.ckpt",
    device="cuda"
)
detox_model.predict(val_df["more_toxic"].tolist()[0])

{'toxicity': 0.0024801511,
 'severe_toxicity': 4.117179e-06,
 'obscene': 0.00016954388,
 'identity_attack': 5.9976803e-05,
 'insult': 0.0008817926,
 'threat': 0.00020323583,
 'sexual_explicit': 0.00015961425}

In [38]:
val_df['more_detoxify_score'] = val_df['more_toxic'].progress_map(lambda line: detox_model.predict(line))
val_df['less_detoxify_score'] = val_df['less_toxic'].progress_map(lambda line: detox_model.predict(line))

more_detoxify_df = val_df["more_detoxify_score"].apply(pd.Series)
less_detoxify_df = val_df["less_detoxify_score"].apply(pd.Series)

display(more_detoxify_df.head())
display(less_detoxify_df.head())

  0%|          | 0/30108 [00:00<?, ?it/s]

  0%|          | 0/30108 [00:00<?, ?it/s]

,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.002480,0.000004,0.000170,0.000060,0.000882,0.000203,0.000160
1,0.938237,0.000397,0.402174,0.003125,0.543495,0.004326,0.550821
2,0.563299,0.000236,0.004133,0.004219,0.006954,0.003256,0.780431
3,0.997367,0.030408,0.922922,0.042344,0.895694,0.128535,0.977353
4,0.924182,0.000020,0.000448,0.024124,0.843042,0.000503,0.000131


,toxicity,severe_toxicity,obscene,identity_attack,insult,threat,sexual_explicit
0,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089


In [39]:
detoxi_df = pd.concat([
    detoxi_df,
    more_detoxify_df.add_prefix("unbiased_albert_more__")
], axis=1)

detoxi_df = pd.concat([
    detoxi_df,
    less_detoxify_df.add_prefix("unbiased_albert_less__")
], axis=1)

detoxi_df.head()

,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,ori_bert_less__severe_toxicity,ori_bert_less__obscene,ori_bert_less__threat,...,unbiased_albert_more__insult,unbiased_albert_more__threat,unbiased_albert_more__sexual_explicit,unbiased_albert_less__toxicity,unbiased_albert_less__severe_toxicity,unbiased_albert_less__obscene,unbiased_albert_less__identity_attack,unbiased_albert_less__insult,unbiased_albert_less__threat,unbiased_albert_less__sexual_explicit
0,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,0.053543,0.924013,0.002496,...,0.000882,0.000203,0.000160,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,0.000096,0.000178,0.000102,...,0.543495,0.004326,0.550821,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,0.000298,0.001521,0.000425,...,0.006954,0.003256,0.780431,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,0.003731,0.048126,0.013924,...,0.895694,0.128535,0.977353,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,0.030403,0.709035,0.002615,...,0.843042,0.000503,0.000131,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    Data Save
</h2>
<br>

In [40]:
detoxi_df = pd.concat([
    val_df,
    detoxi_df
], axis=1)
detoxi_df

,worker,less_toxic,more_toxic,more_detoxify_score,less_detoxify_score,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,...,unbiased_albert_more__insult,unbiased_albert_more__threat,unbiased_albert_more__sexual_explicit,unbiased_albert_less__toxicity,unbiased_albert_less__severe_toxicity,unbiased_albert_less__obscene,unbiased_albert_less__identity_attack,unbiased_albert_less__insult,unbiased_albert_less__threat,unbiased_albert_less__sexual_explicit
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,"{'toxicity': 0.0024801511, 'severe_toxicity': ...","{'toxicity': 0.8043991, 'severe_toxicity': 7.0...",0.916185,0.005880,0.373492,0.002406,0.206716,...,0.000882,0.000203,0.000160,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,"{'toxicity': 0.93823665, 'severe_toxicity': 0....","{'toxicity': 0.015110845, 'severe_toxicity': 2...",0.812823,0.012391,0.393148,0.006643,0.609737,...,0.543495,0.004326,0.550821,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...","{'toxicity': 0.56329876, 'severe_toxicity': 0....","{'toxicity': 0.3069172, 'severe_toxicity': 4.2...",0.220851,0.001472,0.135154,0.000945,0.012083,...,0.006954,0.003256,0.780431,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,"{'toxicity': 0.99736696, 'severe_toxicity': 0....","{'toxicity': 0.047373112, 'severe_toxicity': 7...",0.998102,0.291183,0.984713,0.004768,0.951937,...,0.895694,0.128535,0.977353,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist","{'toxicity': 0.9241818, 'severe_toxicity': 2.0...","{'toxicity': 0.9656493, 'severe_toxicity': 0.0...",0.962041,0.018535,0.088418,0.009725,0.431338,...,0.843042,0.000503,0.000131,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30103,461,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,"{'toxicity': 0.9538608, 'severe_toxicity': 0.0...","{'toxicity': 0.62931985, 'severe_toxicity': 5....",0.994799,0.193718,0.968026,0.015628,0.287761,...,0.081581,0.046983,0.980279,0.629320,0.000054,0.377633,0.001586,0.229485,0.000809,0.001818
30104,527,I'm sorry. I'm not an admin. I will give you t...,get out my large penis,"{'toxicity': 0.9538608, 'severe_toxicity': 0.0...","{'toxicity': 0.62931985, 'severe_toxicity': 5....",0.994799,0.193718,0.968026,0.015628,0.287761,...,0.081581,0.046983,0.980279,0.629320,0.000054,0.377633,0.001586,0.229485,0.000809,0.001818
30105,352,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,"{'toxicity': 0.6102418, 'severe_toxicity': 4.4...","{'toxicity': 0.03797424, 'severe_toxicity': 3....",0.942232,0.016445,0.745451,0.001299,0.642263,...,0.464856,0.007979,0.007021,0.037974,0.000004,0.000145,0.000162,0.040664,0.000172,0.000071
30106,311,"wow...\nare you out of your mind, how was my e...",Piss off you slant eyed-gook,"{'toxicity': 0.6102418, 'severe_toxicity': 4.4...","{'toxicity': 0.03797424, 'severe_toxicity': 3....",0.942232,0.016445,0.745451,0.001299,0.642263,...,0.464856,0.007979,0.007021,0.037974,0.000004,0.000145,0.000162,0.040664,0.000172,0.000071


In [41]:
detoxi_df = detoxi_df.drop(["more_detoxify_score", "less_detoxify_score"], axis=1)
detoxi_df.head()

,worker,less_toxic,more_toxic,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,...,unbiased_albert_more__insult,unbiased_albert_more__threat,unbiased_albert_more__sexual_explicit,unbiased_albert_less__toxicity,unbiased_albert_less__severe_toxicity,unbiased_albert_less__obscene,unbiased_albert_less__identity_attack,unbiased_albert_less__insult,unbiased_albert_less__threat,unbiased_albert_less__sexual_explicit
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,...,0.000882,0.000203,0.000160,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,...,0.543495,0.004326,0.550821,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,...,0.006954,0.003256,0.780431,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,...,0.895694,0.128535,0.977353,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,...,0.843042,0.000503,0.000131,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089


In [42]:
detoxi_df.to_csv("../data/interim/021_exp/all_jigsaw_pred_label.csv", index=False)

<br>
<h1 style = "font-size:45px; font-family:Comic Sans MS ; font-weight : normal; background-color: #4c1c84 ; color : #eeebf1; text-align: center; border-radius: 100px 100px;">
   Weight Optimize
</h1>
<br>

In [44]:
with timer("detoxify load"):
    detoxi_df = pd.read_csv("../data/interim/021_exp/all_jigsaw_pred_label.csv")
    display(detoxi_df.head())

<< detoxify load >> Start


,worker,less_toxic,more_toxic,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,...,unbiased_albert_more__insult,unbiased_albert_more__threat,unbiased_albert_more__sexual_explicit,unbiased_albert_less__toxicity,unbiased_albert_less__severe_toxicity,unbiased_albert_less__obscene,unbiased_albert_less__identity_attack,unbiased_albert_less__insult,unbiased_albert_less__threat,unbiased_albert_less__sexual_explicit
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,...,0.000882,0.000203,0.000160,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,...,0.543495,0.004326,0.550821,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,...,0.006954,0.003256,0.780431,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,...,0.895694,0.128535,0.977353,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,...,0.843042,0.000503,0.000131,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089


<< detoxify load >> 8.0GB(+0.0GB):0.7sec


In [45]:
more_list = [col for col in detoxi_df.columns.tolist() if '_more__' in col]
less_list = [col for col in detoxi_df.columns.tolist() if '_less__' in col]
print(len(more_list), len(less_list), detoxi_df.shape[1])

33 33 69


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    何も考えずSum
</h2>
<br>

In [48]:
OOF_DF = pd.DataFrame()
OOF_DF["more_total_score"] = detoxi_df[more_list].sum(axis=1)
OOF_DF["less_total_score"] = detoxi_df[less_list].sum(axis=1)
OOF_DF["detoxify_diff"] = OOF_DF["more_total_score"] - OOF_DF["less_total_score"]
detoxify_score = OOF_DF[OOF_DF["detoxify_diff"]>0]["detoxify_diff"].count()/len(OOF_DF)
print(f"Sum Detoxify Model Score: {detoxify_score:.6f}")

Sum Detoxify Model Score: 0.701309


<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    LinearRegression
</h2>
<br>

In [183]:
val_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/validation_data.csv")
test_df = pd.read_csv("/mnt/work/data/kaggle/Jigsaw/comments_to_score.csv")

display(val_df.head())
display(test_df.head())

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [184]:
detoxi_df.head()

,worker,less_toxic,more_toxic,ori_bert_more__toxicity,ori_bert_more__severe_toxicity,ori_bert_more__obscene,ori_bert_more__threat,ori_bert_more__insult,ori_bert_more__identity_attack,ori_bert_less__toxicity,...,unbiased_albert_more__insult,unbiased_albert_more__threat,unbiased_albert_more__sexual_explicit,unbiased_albert_less__toxicity,unbiased_albert_less__severe_toxicity,unbiased_albert_less__obscene,unbiased_albert_less__identity_attack,unbiased_albert_less__insult,unbiased_albert_less__threat,unbiased_albert_less__sexual_explicit
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...,0.916185,0.005880,0.373492,0.002406,0.206716,0.002737,0.987555,...,0.000882,0.000203,0.000160,0.804399,0.000070,0.354872,0.002732,0.322802,0.001026,0.012820
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...,0.812823,0.012391,0.393148,0.006643,0.609737,0.074555,0.001304,...,0.543495,0.004326,0.550821,0.015111,0.000002,0.000134,0.000118,0.012538,0.000030,0.000035
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu...",0.220851,0.001472,0.135154,0.000945,0.012083,0.001916,0.071497,...,0.006954,0.003256,0.780431,0.306917,0.000004,0.000093,0.045684,0.098857,0.000301,0.000043
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...,0.998102,0.291183,0.984713,0.004768,0.951937,0.021853,0.933284,...,0.895694,0.128535,0.977353,0.047373,0.000007,0.000818,0.001216,0.029153,0.001869,0.000303
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist",0.962041,0.018535,0.088418,0.009725,0.431338,0.506032,0.972585,...,0.843042,0.000503,0.000131,0.965649,0.002896,0.123649,0.047530,0.145983,0.021697,0.977089


In [188]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(fit_intercept=False, normalize=False)
lr.fit(detoxi_df[more_list].values - detoxi_df[less_list].values, np.zeros(len(detoxi_df))+1)
param_list = lr.coef_

In [189]:
OOF_DF = pd.DataFrame()
more_score = []
less_score = []
detoxi_df[more_list[0]]*param_list[0]
for i, (more_col, less_col) in enumerate(zip(more_list, less_list)):
    OOF_DF[f"fix_{i}"] = (detoxi_df[more_col].values-detoxi_df[less_col].values)*param_list[i]
    
OOF_DF["fix_total_score"] = OOF_DF[[col for col in OOF_DF.columns if "fix_" in col]].sum(axis=1)

detoxify_score = OOF_DF[OOF_DF["fix_total_score"]>0]["fix_total_score"].count()/len(OOF_DF)
print(f"Sum Detoxify Model Score: {detoxify_score:.6f}")

Sum Detoxify Model Score: 0.703999


In [190]:
for i in range(len(param_list)):
    print(f"{more_list[i]} |{param_list[i]}")

ori_bert_more__toxicity |0.09424351067249925
ori_bert_more__severe_toxicity |0.3963742025272966
ori_bert_more__obscene |-0.0029819106660735296
ori_bert_more__threat |-0.21914504316723296
ori_bert_more__insult |-0.04599030198530798
ori_bert_more__identity_attack |-0.10720589042411185
unbiased_roberta_more__toxicity |0.06191321697070442
unbiased_roberta_more__severe_toxicity |-0.32246482212845434
unbiased_roberta_more__obscene |0.022537712428540354
unbiased_roberta_more__identity_attack |0.32842680856091533
unbiased_roberta_more__insult |0.08425011218569295
unbiased_roberta_more__threat |0.40350084841259864
unbiased_roberta_more__sexual_explicit |0.1026037893421568
multilingual_xlmroberta_more__toxicity |0.11075857703948012
multilingual_xlmroberta_more__severe_toxicity |0.2857087641539771
multilingual_xlmroberta_more__obscene |-0.009078136067087314
multilingual_xlmroberta_more__identity_attack |-0.04033293509194361
multilingual_xlmroberta_more__insult |-0.009062917497993917
multilingual_

<br>
<h2 style = "font-size:45px;
             font-family:Comic Sans MS ;
             font-weight : normal; 
             background-color: #eeebf1 ;
             color : #4c1c84;
             text-align: center;
             border-radius: 100px 100px;">
    IRT
</h2>
<br>

In [194]:
data_df = pd.read_csv("https://raw.githubusercontent.com/gagolews/ordinal_regression_data/master/cement_strength.csv")
data_df["response"] = data_df["response"]-1 # labels should start at 0

data_labels = data_df["response"]
data_features = data_df.loc[:, ["V1", "V2", "V3", "V4", "V5", "V6", "V7", "V8"]]

print('Number of features:', data_features.shape[1])
print('Number of examples:', data_features.shape[0])
print('Labels:', np.unique(data_labels.values))

Number of features: 8
Number of examples: 998
Labels: [0 1 2 3 4]
